### Importing libraries

In [1]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt

### Data read and cleanup

In [2]:
train = pd.read_csv('lab6_data.csv')
# I think we can skip those
train = train.drop(["PassengerId", "Ticket","Cabin","Name"],axis=1)

# one-hot encoding non numeric columns
train = pd.get_dummies(train)

# fill nans
nan_age = train[train.Age.apply(np.isnan)]
age = train[~train.Age.apply(np.isnan)]
est = GradientBoostingRegressor(n_estimators=400, learning_rate=0.1, max_depth=1, loss='huber')
est = est.fit(age.drop("Age",axis=1), age["Age"])
nan_age["Age"] = est.predict(nan_age.drop("Age",axis=1))
train = age.append(nan_age)
train.head(10)

/home/dev/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
6,0,1,54.0,0,0,51.8625,0,1,0,0,1
7,0,3,2.0,3,1,21.0750,0,1,0,0,1
8,1,3,27.0,0,2,11.1333,1,0,0,0,1
9,1,2,14.0,1,0,30.0708,1,0,1,0,0
10,1,3,4.0,1,1,16.7000,1,0,0,0,1


### Splitting data

In [3]:
data, target = train.drop("Survived",axis=1), train['Survived']
(X_train, X_test, y_train, y_test) = train_test_split(data, target, test_size=0.2)

### Checking sklearn prediction

In [4]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)
y_score = tree_clf.predict(X_test)
# export_graphviz(tree_clf, out_file='tree.dot') 
roc_auc_score(y_test, y_score)

0.7664092664092664

### Decision tree implementation

In [5]:
class MyDecisionTree(BaseEstimator, ClassifierMixin):
    class Node:
        def __init__(self, column_name, column_value, result, left = None, right = None):
            self.column_name = column_name
            self.column_value = column_value
            self.result = result
            self.left = left
            self.right = right
        
    def fit(self, X, y):
        self.tree = self.__split(X, y)
        return self

    def predict(self, X):
        result = []
        for row in range(0, len(X)):
            result.append(self.__predict(X, row, self.tree))
        return result
    
    def visualize(self):
        self.__visualize(0, 'root', '', '', self.tree)
        
    def __split(self, X, y):
        best_val, best_column, best_cost = None, None, 1000
        for col in X.columns:
            val, cost = self.__split_column(X, y, col)
            if best_cost > cost:
                best_cost = cost
                best_column = col
                best_val = val

        left_node, right_node = self.__split_by_value(X, best_column, best_val)
        leftX, lefty = X[left_node].drop(best_column, axis=1), y[left_node]
        rightX, righty = X[right_node].drop(best_column, axis=1), y[right_node]
        
        if len(leftX) > 0 and len(rightX) > 0:
            l = self.__split(leftX, lefty)
            r = self.__split(rightX, righty)
            return MyDecisionTree.Node(
                best_column, 
                best_val,
                None,
                l,
                r
            )
        elif len(leftX) > 0:
            return MyDecisionTree.Node(best_column, best_val, lefty.ravel()[0])
        else: 
            return MyDecisionTree.Node(best_column, best_val, righty.ravel()[0])
       
    def __split_column(self, X, y, column_name):
        best_value, best_cost = None, 1000
        for val in self.__enumerate_split_points(X[column_name]):
            cost = self.__calc_cost_for_value(X, y, column_name, val)
            
            # let's be greedy
            if cost == 0:
                return (val, cost)
            elif cost < best_cost:
                best_cost = cost
                best_value = val
        return (best_value, best_cost)
    
    def __enumerate_split_points(self, col):
        unique = col.unique().ravel()
        unique = np.sort(unique) 
        if len(unique) == 1:
            yield unique[0]
        else:
            for index in range(1, len(unique)):
                prev = unique[index - 1]
                current = unique[index]
                yield (prev + current)/2
            
    def __calc_cost_for_value(self, X, y, column_name, value):
        left_node, right_node = self.__split_by_value(X, column_name, value)
        left, right = X[left_node], X[right_node]
        
        ll, lr = y[left_node][y == 1], y[left_node][y == 0]
        rl, rr = y[right_node][y == 1], y[right_node][y == 0]
        
        lgini = self.__gini_index(np.array([len(ll), len(lr)]))
        rgini = self.__gini_index(np.array([len(rl), len(rr)]))

        total = len(left) + len(right)
        return lgini*len(left)/total + rgini*len(right)/total
    
    def __split_by_value(self, X, column, value):
        return (X[column] <= value, X[column] > value)
        
    def __gini_index(self, splits):
        total = sum(splits)
        if total == 0:
            return 0
        return 1 - sum((splits/total)**2)
    
    def __visualize(self, depth, text, sign, value, node):
        if node is None:
            return
        
        res = '' if node.result is None else '(' + str(node.result) + ')'
        print('--'*depth, str(depth) + ')', text, sign, value, res)
        self.__visualize(depth + 1, node.column_name, '<=', node.column_value, node.left)
        self.__visualize(depth + 1, node.column_name, '>', node.column_value, node.right)
        
    def __predict(self, X, row, node):
        if node.result is not None:
            return node.result

        if X.iloc[row][node.column_name] <= node.column_value:
            return self.__predict(X, row, node.left)
        
        return self.__predict(X, row, node.right)

### Checking my prediction

In [6]:
tree_clf = MyDecisionTree()
tree_clf.fit(X_train, y_train)
tree_clf.visualize()
y_score = tree_clf.predict(X_test)
roc_auc_score(y_test, y_score)

 0) root   
-- 1) Sex_female <= 0.5 
---- 2) Age <= 3.5 
------ 3) SibSp <= 2.5 
-------- 4) Pclass <= 1.5 (1)
-------- 4) Pclass > 1.5 
---------- 5) Parch <= 1.5 
------------ 6) Fare <= 15.2 (1)
------------ 6) Fare > 15.2 (1)
---------- 5) Parch > 1.5 
------------ 6) Fare <= 24.7875 (1)
------------ 6) Fare > 24.7875 (1)
------ 3) SibSp > 2.5 
-------- 4) Parch <= 1.5 (0)
-------- 4) Parch > 1.5 
---------- 5) Fare <= 39.14375 (1)
---------- 5) Fare > 39.14375 (0)
---- 2) Age > 3.5 
------ 3) Pclass <= 1.5 
-------- 4) Fare <= 26.14375 (0)
-------- 4) Fare > 26.14375 
---------- 5) Embarked_S <= 0.5 
------------ 6) SibSp <= 0.5 
-------------- 7) Parch <= 1.5 (0)
-------------- 7) Parch > 1.5 (0)
------------ 6) SibSp > 0.5 
-------------- 7) Embarked_C <= 0.5 (0)
-------------- 7) Embarked_C > 0.5 
---------------- 8) Parch <= 0.5 (0)
---------------- 8) Parch > 0.5 (1)
---------- 5) Embarked_S > 0.5 
------------ 6) SibSp <= 2.5 
-------------- 7) Parch <= 3.0 (0)
-------------

0.7318532818532818